In [4]:
#%reset
# DERIVE THE EOMs SYMBOLICALLY ----------------------------------------------------------------------------------------------

# import libraries
import sympy as sym
import numpy as np

sym.init_printing()
from IPython.display import display #for pretty printing

from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

In [6]:
# create symbolic variables

# system parameters
g = sym.symbols('g')
mb,ml1l,ml2l,ml1r,ml2r = sym.symbols(['m_{body}','m_{leg1l}','m_{leg2l}','m_{leg1r}','m_{leg2r}']) # mass
lb,ll1l,ll2l,ll1r,ll2r = sym.symbols(['l_{body}','l_{leg1l}','l_{leg2l}','l_{leg1r}','l_{leg2r}']) # length
Inb,Inl1l,Inl2l,Inl1r,Inl2r = sym.symbols(['I_{body}','I_{leg1l}','I_{leg2l}','I_{leg1r}','I_{leg2r}']) # moment of intertia

# generalized coordinates
x,y,thb,thll,thlr,rl,rr = sym.symbols(['x','y','\\theta_{body}','\\theta_{legl}','\\theta_{legr}','r_{left}','r_{right}']) 
dx,dy,dthb,dthll,dthlr,drl,drr = sym.symbols(['\dot{x}','\dot{y}','\dot{\\theta}_{body}','\dot{\\theta}_{legl}','\dot{\\theta}_{legr}','\dot{r}_{left}','\dot{r}_{right}'])
ddx,ddy,ddthb,ddthll,ddthlr,ddrl,ddrr = sym.symbols(['\ddot{x}','\ddot{y}','\ddot{\\theta}_{body}','\ddot{\\theta}_{legl}','\ddot{\\theta}_{legr}','\ddot{r}_{left}','\ddot{r}_{right}']) 

q = sym.Matrix([[x],[y],[thb],[thll],[thlr],[rl],[rr]])
dq = sym.Matrix([[dx],[dy],[dthb],[dthll],[dthlr],[drl],[drr]])
ddq = sym.Matrix([[ddx],[ddy],[ddthb],[ddthll],[ddthlr],[ddrl],[ddrr]])

# forces
# total joint action = actuator + rebound, but that will be dealt with elsewhere
Fl,taul,GRFxl,GRFyl = sym.symbols(['F_{left}','\\tau_{left}','G_x_{left}','G_y_{left}'])
Fr,taur,GRFxr,GRFyr = sym.symbols(['F_{right}','\\tau_{right}','G_x_{right}','G_y_{right}'])

# STEP 1: position vectors ri = [x,y,theta] (world frame)
rb = sym.Matrix([[x],
                [y],
                [thb]])

rl1l = sym.Matrix([[x + 0.5*ll1l*sym.sin(thb + thll)],
                [y - 0.5*ll1l*sym.cos(thb + thll)],
                [thb + thll]])

rl2l = sym.Matrix([[x + (0.5*ll1l+rl)*sym.sin(thb + thll)],
                [y - (0.5*ll1l+rl)*sym.cos(thb + thll)],
                [thb + thll]])
                
rl1r = sym.Matrix([[x + (0.5*ll1r)*sym.sin(thb + thlr)],
                [y - (0.5*ll1r)*sym.cos(thb + thlr)],
                [thb + thlr]])
                
rl2r = sym.Matrix([[x + (0.5*ll1r+rr)*sym.sin(thb + thlr)],
                [y - (0.5*ll1r+rr)*sym.cos(thb + thlr)],
                [thb + thlr]])

# the Jacobians
Jb = rb.jacobian(q)
Jl1l = rl1l.jacobian(q)
Jl2l = rl2l.jacobian(q)
Jl1r = rl1r.jacobian(q)
Jl2r = rl2r.jacobian(q)
                
# STEP 2: generate expressions for the system space velocities from the jacobians
vb = Jb*dq
vl1l = Jl1l*dq
vl2l = Jl2l*dq
vl1r = Jl1r*dq
vl2r = Jl2r*dq
                     
# STEP 3: generate expressions for the kinetic and potential energy
# mass vectors
Mb = sym.Matrix([[mb,mb,Inb]])
Ml1l = sym.Matrix([[ml1l,ml1l,Inl1l]])
Ml2l = sym.Matrix([[ml2l,ml2l,Inl2l]])
Ml1r = sym.Matrix([[ml1r,ml1r,Inl1r]])
Ml2r = sym.Matrix([[ml2r,ml2r,Inl2r]])

T = 0.5*Mb*sym.matrix_multiply_elementwise(vb,vb) + 0.5*Ml1l*sym.matrix_multiply_elementwise(vl1l,vl1l) + 0.5*Ml2l*sym.matrix_multiply_elementwise(vl2l,vl2l) + 0.5*Ml1r*sym.matrix_multiply_elementwise(vl1r,vl1r)+ 0.5*Ml2r*sym.matrix_multiply_elementwise(vl2r,vl2r)
T = T[0]
V = mb*g*rb[1] + ml1l*g*rl1l[1] + ml2l*g*rl2l[1] + ml1r*g*rl1r[1] + ml2r*g*rl2r[1]


M=T.jacobian(dq)
M=M.transpose()
M=M.jacobian(dq)              

C  = sym.zeros(len(q),len(q))                                                         
for i in range(len(q)):                                             
    for j in range(len(q)):
        for n in range(len(q)):
            C[i,j] = C[i,j]+ 0.5*(sym.diff(M[i,j],q[n]) + sym.diff(M[i,n],q[j]) - sym.diff(M[j,n],q[i]))*dq[n];

G  = sym.zeros(len(q),1)                                         
for i in range(len(q)):
    G[i] = sym.diff(V,q[i]); 


In [ ]:
# Lambdify

func_map = {'sin':sin, 'cos':cos} 

sym_list = [g,mb,ml1,ml2,lb,ll1,ll2,Inb,Inl1,Inl2,
            x,y,thb,thl,r,
            dx,dy,dthb,dthl,dr,
            ddx,ddy,ddthb,ddthl,ddr,
            F,tau,GRFx,GRFy]
            
lambEOM_x = sym.lambdify(sym_list,EOMs[0],modules = [func_map])
lambEOM_y = sym.lambdify(sym_list,EOMs[1],modules = [func_map])
lambEOM_thb = sym.lambdify(sym_list,EOMs[2],modules = [func_map])
lambEOM_thl = sym.lambdify(sym_list,EOMs[3],modules = [func_map])
lambEOM_r = sym.lambdify(sym_list,EOMs[4],modules = [func_map])

In [ ]:
# rerun from here if you don't want to calculate the EOM's again
if 'm' in globals():
    del m # deletes the model
    
m = ConcreteModel()

# SETS-----------------------------------------------------------------------------------------------------------------------

N = 100
m.N = RangeSet(N) 

links = [('body',1),('leg',1),('leg',2)]
m.L = Set(dimen=2, initialize = links)

DOFs = ['x','y','theta_b','theta_l','r'] # generalized coordinates
m.DOF = Set(initialize = DOFs) 

# PARAMETERS-----------------------------------------------------------------------------------------------------------------

m.g = Param(initialize = 9.81)

def get_m(n, lb, ln):
    if lb == 'body':
        return 5.0
    else: return 2.5
m.m = Param(m.L, initialize = get_m) # mass of links

def get_len(n, lb, ln):
    if lb == 'body':
        return 1.0
    else: return 0.5
m.len = Param(m.L, initialize = get_len) # length of links

def calculate_In(m, lb, ln): 
    l = (lb,ln)
    # yes, that does mean you have to rebuild the tuple inside the function. Yes, it is dumb.
    return m.m[l]*m.len[l]**2/12 
m.In = Param(m.L, initialize = calculate_In) # moment of inertia

mbody = sum(m.m[l] for l in links)
BW = mbody*m.g.value

# VARIABLES -----------------------------------------------------------------------------------------------------------------

# system coordinates
m.q = Var(m.N, m.DOF) # position
m.dq = Var(m.N, m.DOF) # velocity
m.ddq = Var(m.N, m.DOF) # acceleration

# bound variables
for n in range(1,N+1):
    for l in links:
        m.q[n,'y'].setlb(0.0)

In [ ]:
# TIME AND INTEGRATION

# variable timestep
hm = 0.02 # master timestep
m.h = Var(m.N, bounds = (0.8,1.2))

# Integration constraints 
def BwEuler_p(m,n,dof): # for positions
    if n > 1:
        return m.q[n,dof] == m.q[n-1,dof] + hm*m.h[n]*m.dq[n,dof]
    else:
        return Constraint.Skip #use this to leave out members of a set that the constraint doesn't apply to
m.integrate_p = Constraint(m.N, m.DOF, rule = BwEuler_p)

def BwEuler_v(m,n,dof): # for velocities
    if n > 1:
        return m.dq[n,dof] == m.dq[n-1,dof] + hm*m.h[n]*m.ddq[n-1,dof]
    else:
        return Constraint.Skip 
m.integrate_v = Constraint(m.N, m.DOF, rule = BwEuler_v)

In [ ]:
# GROUND INTERACTIONS -------------------------------------------------------------------------------------------------------

# paramters
m.mu = Param(initialize = 1) # friction coefficient

# sign set for positive and negative components
signs = ['ps','ng'] 
m.sgn = Set(initialize = signs)

WDOFs = ['X','Y',"THETA"] # absolute coordinates (see what I mean about switching between frames the whole time...?)
m.WDOF = Set(initialize = WDOFs) 

# variables
m.footp = Var(m.N, m.WDOF, bounds = (0.0,None)) # foot position
m.footv = Var(m.N, m.WDOF, m.sgn, bounds = (0.0,None)) # foot velocity

m.friction_cone = Var(m.N, bounds = (0.0,None))

m.GRF = Var(m.N, m.WDOF, m.sgn, bounds = (0.0,None)) # ground reaction forces

ground_constraints = ['contact','friction','slip_ps','slip_ng'] 
m.ground_constraints = Set(initialize = ground_constraints) # set for indexing ground-related penalties
m.ground_penalty = Var(m.N, m.ground_constraints, bounds = (0.0,None))

# constraints: aux variables
def def_footp(m,n,dof):
    if dof == 'Y':
        L = 0.5*m.len[('leg',1)] + m.q[n,'r'] + 0.5*m.len[('leg',2)] # total leg length
        thA = m.q[n,'theta_b'] + m.q[n,'theta_l'] # absolute leg angle
        return m.footp[n,dof] == m.q[n,'y'] - L*cos(thA)
    else:
        return Constraint.Skip
m.def_footp = Constraint(m.N, m.WDOF, rule = def_footp)

# lambdify the foot velocity
footx = sym.Matrix([x + (0.5*ll1+r+0.5*ll2)*sym.sin(thb + thl)])
footdx = footx.jacobian(q)*dq
footdx = footdx[0].simplify()
lamb_footdx = sym.lambdify(sym_list,footdx,modules = [func_map])

def def_footv(m,n,dof):
    if dof == 'X':
        var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_l'],m.q[n,'r'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_l'],m.dq[n,'r'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_l'],m.ddq[n,'r'],
            0,0,0,0]
        return m.footv[n,dof,'ps']-m.footv[n,dof,'ng'] == lamb_footdx(*var_list)
    else:
        return Constraint.Skip
m.def_footv = Constraint(m.N, m.WDOF, rule = def_footv)

def def_friction_cone(m,n):
    return m.friction_cone[n] == m.mu*m.GRF[n,'Y','ps'] - (m.GRF[n,'X','ps'] + m.GRF[n,'X','ng'])
m.def_friction_cone = Constraint(m.N, rule = def_friction_cone)

# constraints: complementarity

# contact
def ground_contact(m,n):
    if n < N:
        return m.ground_penalty[n,'contact'] == m.footp[n+1,'Y']*m.GRF[n,'Y','ps'] 
    else:
        return Constraint.Skip
m.ground_contact = Constraint(m.N, rule = ground_contact)

# friction
def ground_friction(m,n):
    return m.ground_penalty[n,'friction'] == (m.footv[n,'X','ps']+m.footv[n,'X','ng'])*m.friction_cone[n]
m.ground_friction = Constraint(m.N, rule = ground_friction)

# slipping
def ground_slip_ps(m,n):
    return m.ground_penalty[n,'slip_ps'] == m.footv[n,'X','ps']*m.GRF[n,'X','ps']
m.ground_slip_ps = Constraint(m.N, rule = ground_slip_ps)

def ground_slip_ng(m,n):
    return m.ground_penalty[n,'slip_ng'] == m.footv[n,'X','ng']*m.GRF[n,'X','ng']
m.ground_slip_ng = Constraint(m.N, rule = ground_slip_ng)

# bound contact forces at last node
for dof in WDOFs:
    for sgn in signs:
        m.GRF[N,dof,sgn].value = 0
        m.GRF[N,dof,sgn].fixed = True

In [ ]:
# HARD JOINT STOPS ----------------------------------------------------------------------------------------------------------

# sets
joints = ['hip','knee']
m.J = Set(initialize = joints)

joint_constraints = ['up','lo'] # set of joint penalties
m.joint_constraints = Set(initialize = joint_constraints)

# parameters
hip_bound = [-np.pi/2,np.pi/2]
m.hip_bound = Param(m.joint_constraints, initialize = {'up':hip_bound[1],'lo':hip_bound[0]}) 

knee_bound = [0.0,0.5]
m.knee_bound = Param(m.joint_constraints, initialize = {'up':knee_bound[1],'lo':knee_bound[0]})

# we can bound the joint coordinates directly
for n in range(1,N+1):
    m.q[n,'theta_l'].setlb(hip_bound[0])
    m.q[n,'theta_l'].setub(hip_bound[1])
    m.q[n,'r'].setlb(knee_bound[0])
    m.q[n,'r'].setub(knee_bound[1])

# variables
m.tau_a = Var(m.N, bounds = (-2,2)) # actuator torque at hip
m.tau_r = Var(m.N, m.sgn, bounds = (0.0,None)) # rebound torque

m.F_a = Var(m.N, bounds = (-3,3)) # actuator force at knee
m.F_r = Var(m.N, m.sgn, bounds = (0.0,None)) # rebound force (acts parallel to the leg)

m.joint_penalty = Var(m.N, m.J, m.joint_constraints, bounds = (0.0,None))

# complementarity
def hip_limits(m,n,jc):
    if n < N:
        if jc == 'up':
            # NEXT angle
            return m.joint_penalty[n,'hip',jc] == (m.hip_bound['up'] - m.q[n+1,'theta_l'])*m.tau_r[n,'ng']
        else:
            return m.joint_penalty[n,'hip',jc] == (m.q[n+1,'theta_l'] - m.hip_bound['lo'])*m.tau_r[n,'ps']
    else:
        return Constraint.Skip
m.hip_limits = Constraint(m.N, m.joint_constraints, rule = hip_limits)

def knee_limits(m,n,jc):
    if n < N:
        if jc == 'up':
            # NEXT distance
            return m.joint_penalty[n,'knee',jc] == (m.knee_bound['up'] - m.q[n+1,'r'])*m.F_r[n,'ng']
        else:
            return m.joint_penalty[n,'knee',jc] == (m.q[n+1,'r'] - m.knee_bound['lo'])*m.F_r[n,'ps']
    else:
        return Constraint.Skip
m.knee_limits = Constraint(m.N, m.joint_constraints, rule = knee_limits)

# bound contact forces at last node
for sgn in signs:
    m.F_r[N,sgn].value = 0
    m.F_r[N,sgn].fixed = True

for sgn in signs:
    m.tau_r[N,sgn].value = 0
    m.tau_r[N,sgn].fixed = True

In [ ]:
# COST FUNCTION -------------------------------------------------------------------------------------------------------------

# minimum time

def CostFun(m):
    T = sum(m.h[n] for n in range(1,N+1))
    penalty_sum = 0
    for n in range(1,N+1):
        for gc in ground_constraints:
            penalty_sum += m.ground_penalty[n,gc]
        for jc in joint_constraints:
            for j in joints:
                penalty_sum += m.joint_penalty[n,j,jc]
    return T+1000*penalty_sum
m.Cost = Objective(rule = CostFun)

In [ ]:
# EQUATIONS OF MOTION -------------------------------------------------------------------------------------------------------
S = BW

def EOM_x(m,n):
    F_in = S*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_in = S*(m.tau_a[n] + m.tau_r[n,'ps'] - m.tau_r[n,'ng'])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_l'],m.q[n,'r'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_l'],m.dq[n,'r'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_l'],m.ddq[n,'r'],
            F_in,tau_in,Gx_in,Gy_in]
    return lambEOM_x(*var_list) == 0
m.EOM_x = Constraint(m.N, rule = EOM_x)

def EOM_y(m,n):
    F_in = S*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_in = S*(m.tau_a[n] + m.tau_r[n,'ps'] - m.tau_r[n,'ng'])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_l'],m.q[n,'r'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_l'],m.dq[n,'r'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_l'],m.ddq[n,'r'],
            F_in,tau_in,Gx_in,Gy_in]
    return lambEOM_y(*var_list) == 0
m.EOM_y = Constraint(m.N, rule = EOM_y)

def EOM_thb(m,n):
    F_in = S*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_in = S*(m.tau_a[n] + m.tau_r[n,'ps'] - m.tau_r[n,'ng'])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_l'],m.q[n,'r'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_l'],m.dq[n,'r'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_l'],m.ddq[n,'r'],
            F_in,tau_in,Gx_in,Gy_in]
    return lambEOM_thb(*var_list) == 0
m.EOM_thb = Constraint(m.N, rule = EOM_thb)

def EOM_thl(m,n):
    F_in = S*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_in = S*(m.tau_a[n] + m.tau_r[n,'ps'] - m.tau_r[n,'ng'])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_l'],m.q[n,'r'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_l'],m.dq[n,'r'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_l'],m.ddq[n,'r'],
            F_in,tau_in,Gx_in,Gy_in]
    return lambEOM_thl(*var_list) == 0
m.EOM_thl = Constraint(m.N, rule = EOM_thl)

def EOM_r(m,n):
    F_in = S*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_in = S*(m.tau_a[n] + m.tau_r[n,'ps'] - m.tau_r[n,'ng'])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_l'],m.q[n,'r'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_l'],m.dq[n,'r'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_l'],m.ddq[n,'r'],
            F_in,tau_in,Gx_in,Gy_in]
    return lambEOM_r(*var_list) == 0
m.EOM_r = Constraint(m.N, rule = EOM_r)

In [ ]:
# INITIALIZE ----------------------------------------------------------------------------------------------------------------
guide = np.linspace(0,5.0,N)
for n in range(1,N+1):
    m.q[n,'x'].value = guide[n-1] + np.random.uniform(-0.25,0.25)
    m.q[n,'y'].value  = np.random.uniform(0.5,1.1)
    m.q[n,'theta_b'].value  = np.random.uniform(-np.pi/8,np.pi/8)
    m.q[n,'theta_l'].value  = np.random.uniform(-np.pi/4,np.pi/4)
    m.q[n,'r'].value  = np.random.uniform(0.0,0.5)
    
    m.GRF[n,'Y','ps'].value = np.random.uniform(0.0,1)
    m.GRF[n,'X','ps'].value = m.mu.value*m.GRF[n,'Y','ps'].value

In [ ]:
# SPRINT --------------------------------------------------------------------------------------------------------------------
# tax day

# initial condition
m.q[1,'x'].value = 0.0
m.q[1,'theta_b'].value = 0.0
m.q[1,'theta_l'].value = 0.0

m.q[1,'x'].fixed = True
m.q[1,'theta_b'].fixed = True
m.q[1,'theta_l'].fixed = True

m.footp[1,'Y'].value = 0.0
m.footp[1,'Y'].fixed = True

for dof in DOFs:
    m.dq[1,dof].value = 0.0
    m.dq[1,dof].fixed = True
      
# final condition
m.q[N,'x'].setlb(5.0)

m.dq[N,'x'].value = 0.0
m.dq[N,'x'].fixed = 0.0
m.dq[N,'theta_b'].value = 0.0
m.dq[N,'theta_b'].fixed = 0.0
m.dq[N,'theta_l'].value = 0.0
m.dq[N,'theta_l'].fixed = 0.0

m.q[N,'theta_b'].value = 0.0
m.q[N,'theta_l'].value = 0.0
m.q[N,'theta_b'].fixed = True
m.q[N,'theta_l'].fixed = True

m.footp[N,'Y'].value = 0.0
m.footp[N,'Y'].fixed = True
    
#m.pprint()

In [ ]:
# solving
#opt = SolverFactory('ipopt') # standard issue, garden variety ipopt

#opt = SolverFactory('ipopt',executable = 'C:/cygwin64/home/Stacey/CoinIpopt/build/bin/ipopt.exe')
opt = SolverFactory('ipopt') 
#opt.options["linear_solver"] = 'ma86'

# solver options
opt.options["expect_infeasible_problem"] = 'yes'
#pt.options["linear_system_scaling"] = 'none'
#opt.options["mu_strategy"] = "adaptive"
opt.options["print_level"] = 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
opt.options["max_iter"] = 30000 # maximum number of iterations
opt.options["max_cpu_time"] = 600 # maximum cpu time in seconds
opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
    
results = opt.solve(m, tee = True) 

In [ ]:
print(results.solver.status) # tells you if the solver had any errors/ warnings
print(results.solver.termination_condition) # tells you if the solution was (locally) optimal, feasible, or neither.

penalty_sum = 0
for n in range(1,N+1):
    for gc in ground_constraints:
        penalty_sum = penalty_sum + m.ground_penalty[n,gc].value
    for jc in joint_constraints:
        for j in joints:
            penalty_sum += m.joint_penalty[n,j,jc].value

print(penalty_sum)

m.pprint() 

In [ ]:
#animate it

fig1, ax1 = plt.subplots(1,1) #create axes
ax1.set_aspect('equal')

def plot_robot(i,m,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-1,10])
    ax.set_ylim([0,3])
    
    #plot body
    body_xb = m.q[i,'x'].value - 0.5*m.len[('body',1)]*cos(m.q[i,'theta_b'].value)
    body_yb = m.q[i,'y'].value - 0.5*m.len[('body',1)]*sin(m.q[i,'theta_b'].value)
    body_xf = m.q[i,'x'].value + 0.5*m.len[('body',1)]*cos(m.q[i,'theta_b'].value)
    body_yf = m.q[i,'y'].value + 0.5*m.len[('body',1)]*sin(m.q[i,'theta_b'].value)  
    ax.plot([body_xb,body_xf],[body_yb,body_yf],color='xkcd:black')
      
    #plot leg 1
    thA = m.q[i,'theta_b'].value+m.q[i,'theta_l'].value
    leg1_xt = m.q[i,'x'].value
    leg1_yt = m.q[i,'y'].value
    leg1_xb = m.q[i,'x'].value + m.len[('leg',1)]*sin(thA)
    leg1_yb = m.q[i,'y'].value - m.len[('leg',1)]*cos(thA)
    ax.plot([leg1_xt,leg1_xb],[leg1_yt,leg1_yb],color='xkcd:black')
    
    #plot leg 2
    Lt = 0.5*m.len[('leg',1)] + m.q[i,'r'].value - 0.5*m.len[('leg',2)]
    Lb = 0.5*m.len[('leg',1)] + m.q[i,'r'].value + 0.5*m.len[('leg',2)]
    leg2_xt = m.q[i,'x'].value + Lt*sin(thA)
    leg2_yt = m.q[i,'y'].value - Lt*cos(thA)
    leg2_xb = m.q[i,'x'].value + Lb*sin(thA)
    leg2_yb = m.q[i,'y'].value - Lb*cos(thA)
    ax.plot([leg2_xt,leg2_xb],[leg2_yt,leg2_yb],color='xkcd:black')
    
update = lambda i: plot_robot(i,m,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 50,repeat=True)

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook
